In [1]:
import PIL as pillow
%matplotlib inline
import numpy as np
import matplotlib.image as mpimg
from scipy import ndimage
from PIL import Image
from PIL import ImageFilter
from matplotlib import pyplot as plt
from matplotlib.pyplot import *
import matplotlib.image as mpimg
import os
import cv2
import time
import glob
import scipy.fftpack as fp
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
def fft(imagepath):
    ## Functions to go from image to frequency-image and back
    im2freq = lambda data:fp.rfft(fp.rfft(data, axis=0),axis=1)
    freq2im = lambda f:fp.irfft(fp.irfft(f, axis=1),axis=0)


    ## Read in data file and transform
    data = np.array(Image.open(imagepath))
    freq = im2freq(data)
    back = freq2im(freq)
    
    # Make sure the forward and backward transforms work!
    assert(np.allclose(data, back))
    ## Helper functions to rescale a frequency-image to [0, 255] and save
    remmax = lambda x: x/x.max()
    remmin = lambda x: x - np.amin(x, axis=(0,1), keepdims=True)
    touint8 = lambda x: (remmax(remmin(x))*(256-1e-4)).astype(int)
    return(touint8(freq))

In [3]:
def laplacian(path):
    temp_path_image = cv2.imread(path)
    blur = cv2.GaussianBlur(temp_path_image,(3,3),0,0, cv2.BORDER_DEFAULT)
    laplacian = cv2.Laplacian(blur, ddepth = cv2.CV_16UC3 , ksize = 3, scale = 1,delta = 0, borderType=cv2.BORDER_DEFAULT)
    (mean, stddev) = cv2.meanStdDev(laplacian)
    laplacian_operator_std_dev = stddev[0]
    variance = laplacian_operator_std_dev**2
    return(variance)

In [23]:
for f in (glob.glob("/Users/rjosyula/Documents/GitHub/BPC/SEQ/Images/RAW/*")):
    print(f.split("/")[9] +"  " + str(float(laplacian(f))) + "   ")

Raw_Image_1.bmp  1420.83867543   
Raw_Image_10.bmp  1201.46227311   
Raw_Image_11.bmp  1501.81645738   
Raw_Image_12.bmp  1571.1212928   
Raw_Image_13.bmp  1530.28022368   
Raw_Image_14.bmp  1434.39348569   
Raw_Image_15.bmp  1526.56368705   
Raw_Image_2.bmp  1583.39356678   
Raw_Image_3.bmp  1625.31896975   
Raw_Image_4.bmp  1559.90567985   
Raw_Image_5.bmp  1616.40562332   
Raw_Image_6.bmp  1498.12120444   
Raw_Image_7.bmp  1393.99237588   
Raw_Image_8.bmp  1238.55725091   
Raw_Image_9.bmp  1209.50478399   


In [19]:
def variance_of_laplacian(image):
    imagex = cv2.imread(image)
    gray = cv2.cvtColor(imagex, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(gray, cv2.CV_64F).var()
    return fm

In [24]:
for f in (glob.glob("/Users/rjosyula/Documents/GitHub/BPC/SEQ/Images/RAW/*")):
    print(f.split("/")[9] +"  " + str(float(variance_of_laplacian(f))) + "   ")

Raw_Image_1.bmp  147081.498445   
Raw_Image_10.bmp  157803.440692   
Raw_Image_11.bmp  141834.369158   
Raw_Image_12.bmp  141791.99317   
Raw_Image_13.bmp  144846.177538   
Raw_Image_14.bmp  154332.569588   
Raw_Image_15.bmp  152382.608398   
Raw_Image_2.bmp  140117.332741   
Raw_Image_3.bmp  144037.971754   
Raw_Image_4.bmp  144689.098811   
Raw_Image_5.bmp  143634.675249   
Raw_Image_6.bmp  148133.912326   
Raw_Image_7.bmp  141556.850355   
Raw_Image_8.bmp  153347.528223   
Raw_Image_9.bmp  155932.936015   


In [33]:
def secondfft(image):
    imblur = scipy.misc.imfilter.convolve(image, blurfilter)
    blurfilter = (1./273.) * np.array([[1,4,7,4,1],[4,16,26,16,4],[7,26,41,26,7],[4,16,26,16,4],[1,4,7,4,1]])
    # compute fft of kernel with padding
    padfilfft = np.fft.fft2(blurfilter, s=im1.shape[:2], axes=(0,1))    

    # take fft of image
    imfft = np.fft.fft2(imblur)

    # element division
    deconvolveimfft = np.divide(imfft, padfilfft)
    deconvolveim = np.abs(np.fft.ifft2(deconvolveimfft))
    return imfft

In [34]:
for f in (glob.glob("/Users/rjosyula/Documents/GitHub/BPC/SEQ/Images/RAW/*")):
    print(f.split("/")[9] +"  " + str(secondfft(f)) + "   ")

NameError: global name 'scipy' is not defined

In [40]:
def apply_sobel_filter(input_image):
    im = np.asarray(input_image)
    #im = input_image
    sx = ndimage.sobel(im, axis=0, mode='constant')
    sy = ndimage.sobel(im, axis=1, mode='constant')
    mag = np.hypot(sx, sy)  # magnitude
    mag *= 255.0 / np.max(mag)  # normalize (Q&D)
    sob = np.hypot(sx, sy, dtype=np.float32)
    sob *= 255.0/np.max(sob)
    return sob
for f in (glob.glob("/Users/rjosyula/Downloads/SampleImages/*")):
    cv2.imshow(f)

TypeError: Required argument 'mat' (pos 2) not found